<a href="https://colab.research.google.com/github/plaban1981/Langchain_usecases/blob/main/The_EASIEST_way_to_RUN_Llama2_like_LLMs_on_CPU!!!_GGML_Models_healtchare_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install ctransformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.9 MB/s eta 0:00:00


In [2]:
from ctransformers import AutoModelForCausalLM

llm = AutoModelForCausalLM.from_pretrained('TheBloke/Llama-2-7B-Chat-GGML', model_file = 'llama-2-7b-chat.ggmlv3.q4_K_S.bin' )


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

## Inference

In [3]:
%%time
for word in llm('Write a joke about Elon Musk', stream = True):
  print(word, end='')


 If he were to start a dating app, it would be called "Tesla-Date" and the slogan would be "Charging your love life one match at a time."CPU times: user 40.2 s, sys: 70.4 ms, total: 40.3 s
Wall time: 22.7 s


In [4]:
!pip install -qU langchain openai transformers sentence_transformers faiss_cpu altair accelerate tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.5 MB/s eta 0:00:00


## Import Required Libraries

In [5]:
from langchain.llms.openai import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import CTransformers,OpenAI
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter

##Load the pdf file

In [7]:
! pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.1/271.1 kB 4.2 MB/s eta 0:00:00


In [8]:
loader = PyPDFLoader("/content/mental_health_Document.pdf")
documents = loader.load()

In [9]:
documents

[Document(page_content='See discussions, st ats, and author pr ofiles f or this public ation at : https://www .researchgate.ne t/public ation/255657987\nDeﬁning mental health and mental illness\nArticle  · Januar y 2009\nCITATIONS\n9READS\n141,176\n2 author s, including:\nSome o f the author s of this public ation ar e also w orking on these r elat ed pr ojects:\nCultural P sychiatr y View pr oject\nDiversifying the medic al curriculum  View pr oject\nNisha Dogr a\nUniv ersity of L eicester\n152 PUBLICA TIONS \xa0\xa0\xa02,847  CITATIONS \xa0\xa0\xa0\nSEE PROFILE\nAll c ontent f ollo wing this p age was uplo aded b y Nisha Dogr a on 20 May 2014.\nThe user has r equest ed enhanc ement of the do wnlo aded file.', metadata={'source': '/content/mental_health_Document.pdf', 'page': 0}),
 Document(page_content='Deﬁning mental health and\nmental illness\nSharon Leighton and Nisha Dogra\nKey features \x7fDiscussion of the terminological confusion that exists in relation to issues\nassociated w

##Split text into chunks

In [10]:
text_splitter =  RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
#
text_chunks = text_splitter.split_documents(documents)

In [11]:
text_chunks

[Document(page_content='See discussions, st ats, and author pr ofiles f or this public ation at : https://www .researchgate.ne t/public ation/255657987\nDeﬁning mental health and mental illness\nArticle  · Januar y 2009\nCITATIONS\n9READS\n141,176\n2 author s, including:\nSome o f the author s of this public ation ar e also w orking on these r elat ed pr ojects:\nCultural P sychiatr y View pr oject\nDiversifying the medic al curriculum  View pr oject\nNisha Dogr a\nUniv ersity of L eicester\n152 PUBLICA TIONS \xa0\xa0\xa02,847  CITATIONS \xa0\xa0\xa0\nSEE PROFILE\nAll c ontent f ollo wing this p age was uplo aded b y Nisha Dogr a on 20 May 2014.\nThe user has r equest ed enhanc ement of the do wnlo aded file.', metadata={'source': '/content/mental_health_Document.pdf', 'page': 0}),
 Document(page_content='Deﬁning mental health and\nmental illness\nSharon Leighton and Nisha Dogra\nKey features \x7fDiscussion of the terminological confusion that exists in relation to issues\nassociated w

## Create Text Embeddings

* This is a sentence-transformers model: It maps sentences & paragraphs to a 384 dimensional dense vector space and can be used for tasks like clustering or semantic search.

In [12]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                   model_kwargs={"device":"cpu"})

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [29]:
llm = CTransformers(model = 'TheBloke/Llama-2-7B-Chat-GGML',
                            model_file = 'llama-2-7b-chat.ggmlv3.q4_K_S.bin',
                            model_type="llama",
                            config = {"max_new_tokens":256,"temperature":0.01})

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

## VectorStore

In [23]:
#vectorstore
vector_store = FAISS.from_documents(text_chunks,embeddings)

## Memory

In [18]:
memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)

## Build the converstaional chain

In [30]:
chain = ConversationalRetrievalChain.from_llm(llm=llm,
                                              chain_type="stuff",
                                              retriever=vector_store.as_retriever(search_kwargs={"k":2}),
                                              memory=memory)

In [28]:
chain("What is the document about?")

{'question': 'What is the document about?',
 'chat_history': [HumanMessage(content='What is the document about?', additional_kwargs={}, example=False),
  AIMessage(content=' The document is about stigma related to mental health, specifically how it affects children and young people. It discusses the scale of the problem, access to problem, and access to problem, and access to problem, and access to problem, and access to problem, and access to problem, and access to problem, and access to problem, and access to problem, and access to problem, and access to problem, and access to problem, and access to problem, and access to problem, and access to problem, and access to problem, and access to problem, and access to problem, and access to problem, and access to problem, and access to problem, and access to problem, and access to problem, and access to problem, and access to problem, and access to problem, and access to problem, and access to problem, and access to problem, and access to 

In [31]:
response = chain("What are the stigmas associated with Mental illness?")

In [32]:
response['answer']

" According to the text, some of the stigmas associated with mental illness include being seen as dangerous, having a poor prognosis, and disrupting normal social interactions. It also mentions that adolescents have negative attitudes towards groups described as deviant, including those with mental illness, which can increase with age.\n\nUnhelpful Answer: I don't know the answer to this question because it is not provided in the text."